In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
#this code is  by akash pal

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/shl-intern-hiring-assessment/dataset/sample_submission.csv
/kaggle/input/shl-intern-hiring-assessment/dataset/train.csv
/kaggle/input/shl-intern-hiring-assessment/dataset/test.csv
/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_885.wav
/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_698.wav
/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_1176.wav
/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_1215.wav
/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_66.wav
/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_386.wav
/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_1026.wav
/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_330.wav
/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_72.wav
/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_858.wav
/kaggle/input/shl-intern-hiring-ass

In [3]:
import numpy as np
import pandas as pd
import os
import librosa
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import pearsonr

# File paths

In [4]:
TRAIN_PATH = '/kaggle/input/shl-intern-hiring-assessment/dataset/train.csv'
TEST_PATH = '/kaggle/input/shl-intern-hiring-assessment/dataset/test.csv'
TRAIN_AUDIO_DIR = '/kaggle/input/shl-intern-hiring-assessment/dataset/audios_train/'
TEST_AUDIO_DIR = '/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/'
SUBMISSION_TEMPLATE = '/kaggle/input/shl-intern-hiring-assessment/dataset/sample_submission.csv'


In [7]:
import numpy as np
import pandas as pd
import os
import librosa
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import pearsonr
import lightgbm as lgb

# File paths
TRAIN_PATH = '/kaggle/input/shl-intern-hiring-assessment/dataset/train.csv'
TEST_PATH = '/kaggle/input/shl-intern-hiring-assessment/dataset/test.csv'
TRAIN_AUDIO_DIR = '/kaggle/input/shl-intern-hiring-assessment/dataset/audios_train/'
TEST_AUDIO_DIR = '/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/'
SUBMISSION_TEMPLATE = '/kaggle/input/shl-intern-hiring-assessment/dataset/sample_submission.csv'

# Load data
def load_data():
    train_df = pd.read_csv(TRAIN_PATH)
    test_df = pd.read_csv(TEST_PATH)
    print(f"Loaded {len(train_df)} training and {len(test_df)} test samples.")
    return train_df, test_df

# Feature extractor
def extract_features(file_path, n_mfcc=20):
    try:
        y, sr = librosa.load(file_path, sr=None)

        # MFCC
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
        mfcc_mean = np.mean(mfcc, axis=1)
        mfcc_std = np.std(mfcc, axis=1)

        # Delta
        delta = librosa.feature.delta(mfcc)
        delta_mean = np.mean(delta, axis=1)
        delta_std = np.std(delta, axis=1)

        # Delta-Delta
        delta2 = librosa.feature.delta(mfcc, order=2)
        delta2_mean = np.mean(delta2, axis=1)
        delta2_std = np.std(delta2, axis=1)

        # Chroma
        chroma = librosa.feature.chroma_stft(y=y, sr=sr)
        chroma_mean = np.mean(chroma, axis=1)
        chroma_std = np.std(chroma, axis=1)

        # Spectral Contrast
        contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
        contrast_mean = np.mean(contrast, axis=1)
        contrast_std = np.std(contrast, axis=1)

        return np.concatenate([mfcc_mean, mfcc_std, delta_mean, delta_std,
                               delta2_mean, delta2_std, chroma_mean, chroma_std,
                               contrast_mean, contrast_std])
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return np.zeros(n_mfcc * 6 + 12 * 2 + 7 * 2)

# Batch extraction
def extract_batch_features(df, directory, is_train=True):
    features = []
    labels = []
    for idx, row in df.iterrows():
        path = os.path.join(directory, row['filename'])
        f = extract_features(path)
        features.append(f)
        if is_train:
            labels.append(row['label'])
        if idx % 50 == 0:
            print(f"Processed {idx}/{len(df)} files")
    return np.array(features), np.array(labels) if is_train else None

# Evaluate
def evaluate(model, X, y):
    preds = model.predict(X)
    mse = mean_squared_error(y, preds)
    r2 = r2_score(y, preds)
    corr, _ = pearsonr(y, preds)
    print(f"MSE: {mse:.4f} | R2: {r2:.4f} | Pearson Corr: {corr:.4f}")
    return preds

# Main
def main():
    train_df, test_df = load_data()

    print("Extracting training features...")
    X_train_full, y_train_full = extract_batch_features(train_df, TRAIN_AUDIO_DIR)

    print("Splitting train/val...")
    X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=42)

    print("Training LightGBM...")
    model = lgb.LGBMRegressor(n_estimators=300, learning_rate=0.05, random_state=42)
    model.fit(X_train, y_train)

    print("Evaluating on validation set:")
    val_preds = evaluate(model, X_val, y_val)

    print("Retraining on full data...")
    model.fit(X_train_full, y_train_full)

    print("Extracting test features...")
    X_test, _ = extract_batch_features(test_df, TEST_AUDIO_DIR, is_train=False)

    print("Predicting on test set...")
    test_preds = model.predict(X_test)
    test_preds = np.clip(test_preds, 0, 100)

    # Submission
    submission = pd.read_csv(SUBMISSION_TEMPLATE)
    submission['label'] = test_preds
    submission.to_csv("submission.csv", index=False)
    print("✅ submission.csv saved.")
    print(submission.head())

main()


Loaded 444 training and 195 test samples.
Extracting training features...
Processed 0/444 files
Processed 50/444 files
Processed 100/444 files
Processed 150/444 files
Processed 200/444 files
Processed 250/444 files
Processed 300/444 files
Processed 350/444 files
Processed 400/444 files
Splitting train/val...
Training LightGBM...
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003621 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 18812
[LightGBM] [Info] Number of data points in the train set: 355, number of used features: 158
[LightGBM] [Info] Start training from score 3.635211
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No fur